## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Unit 6 Challenge, Deliverable 2

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
from pprint import pprint
import numpy as np

# Import Google API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("copy_of_WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kudymkar,RU,59.0173,54.6560,40.75,94,100,10.80,overcast clouds
1,1,Airai,TL,-8.9266,125.4092,64.09,89,99,0.16,overcast clouds
2,2,Chuy,UY,-33.6971,-53.4616,71.19,73,99,14.52,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,46.06,53,0,19.57,clear sky
4,4,Teguldet,RU,57.3064,88.1667,25.20,61,53,10.18,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Saint-Georges,GD,12.0564,-61.7485,85.68,70,20,13.80,few clouds
16,16,Victoria,HK,22.2855,114.1577,76.32,81,49,1.01,light rain
20,20,Muisne,EC,0.6000,-80.0333,75.31,73,80,8.10,broken clouds
23,23,Puro,PH,12.4841,123.3827,80.58,77,87,14.25,overcast clouds
25,25,Banda Aceh,ID,5.5577,95.3222,79.16,81,100,4.07,overcast clouds
26,26,Mahebourg,MU,-20.4081,57.7000,78.26,73,60,5.64,broken clouds
31,31,Ponta Do Sol,PT,32.6667,-17.1000,81.72,75,91,9.93,overcast clouds
32,32,Hermanus,ZA,-34.4187,19.2345,75.79,68,8,13.49,clear sky
35,35,Vanimo,PG,-2.6741,141.3028,80.24,85,64,3.04,broken clouds
37,37,Bredasdorp,ZA,-34.5322,20.0403,75.27,54,41,11.97,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.

In [7]:
preferred_cities_df.count()

City_ID                216
City                   216
Country                213
Lat                    216
Lng                    216
Max Temp               216
Humidity               216
Cloudiness             216
Wind Speed             216
Current Description    216
dtype: int64

In [8]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Saint-Georges,GD,12.0564,-61.7485,85.68,70,20,13.80,few clouds
16,16,Victoria,HK,22.2855,114.1577,76.32,81,49,1.01,light rain
20,20,Muisne,EC,0.6000,-80.0333,75.31,73,80,8.10,broken clouds
23,23,Puro,PH,12.4841,123.3827,80.58,77,87,14.25,overcast clouds
25,25,Banda Aceh,ID,5.5577,95.3222,79.16,81,100,4.07,overcast clouds


In [10]:
# Make sure all empty rows have been dropped
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Saint-Georges,GD,85.68,few clouds,12.0564,-61.7485,
16,Victoria,HK,76.32,light rain,22.2855,114.1577,
20,Muisne,EC,75.31,broken clouds,0.6000,-80.0333,
23,Puro,PH,80.58,overcast clouds,12.4841,123.3827,
25,Banda Aceh,ID,79.16,overcast clouds,5.5577,95.3222,
26,Mahebourg,MU,78.26,broken clouds,-20.4081,57.7000,
31,Ponta Do Sol,PT,81.72,overcast clouds,32.6667,-17.1000,
32,Hermanus,ZA,75.79,clear sky,-34.4187,19.2345,
35,Vanimo,PG,80.24,broken clouds,-2.6741,141.3028,
37,Bredasdorp,ZA,75.27,scattered clouds,-34.5322,20.0403,


In [12]:
# # 6a. Set parameters to search for hotels with 5000 meters.
# params = {
#     "radius": 5000,
#     "type": "lodging",
#     "key": g_key
# }

# # 6c. Get latitude and longitude from DataFrame
# row = hotel_df.loc[hotel_df["City"]=="Cape Town"]


# lat = row["Lat"].values[0]
# lng = row["Lng"].values[0]
# params["location"] = f"{lat},{lng}"
# params
# #lat


# lat = row["Lat"]
# lat

In [13]:

# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# # hotels = requests.get(base_url, params=params).json()
# pprint(hotels)

In [14]:
# hotel_name = hotels["results"][0]["name"]
# hotel_name

In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
 
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
hotel_df.head(10)
#hotel_df.count()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Saint-Georges,GD,85.68,few clouds,12.0564,-61.7485,Radisson Grenada Beach Resort
16,Victoria,HK,76.32,light rain,22.2855,114.1577,Mini Hotel Central
20,Muisne,EC,75.31,broken clouds,0.6000,-80.0333,Quinta Samay
23,Puro,PH,80.58,overcast clouds,12.4841,123.3827,
25,Banda Aceh,ID,79.16,overcast clouds,5.5577,95.3222,Sulthan Hotel International
26,Mahebourg,MU,78.26,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
31,Ponta Do Sol,PT,81.72,overcast clouds,32.6667,-17.1000,Hotel do Campo
32,Hermanus,ZA,75.79,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
35,Vanimo,PG,80.24,broken clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
37,Bredasdorp,ZA,75.27,scattered clouds,-34.5322,20.0403,Bredasdorp Country Manor


In [17]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("", np.nan, inplace = True)
hotel_df.dropna(subset=["Hotel Name"], inplace = True)

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Saint-Georges,GD,85.68,few clouds,12.0564,-61.7485,Radisson Grenada Beach Resort
16,Victoria,HK,76.32,light rain,22.2855,114.1577,Mini Hotel Central
20,Muisne,EC,75.31,broken clouds,0.6000,-80.0333,Quinta Samay
25,Banda Aceh,ID,79.16,overcast clouds,5.5577,95.3222,Sulthan Hotel International
26,Mahebourg,MU,78.26,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
31,Ponta Do Sol,PT,81.72,overcast clouds,32.6667,-17.1000,Hotel do Campo
32,Hermanus,ZA,75.79,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
35,Vanimo,PG,80.24,broken clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
37,Bredasdorp,ZA,75.27,scattered clouds,-34.5322,20.0403,Bredasdorp Country Manor
39,Calamar,CO,84.15,overcast clouds,10.2511,-74.9146,Hostal Katyalex


In [18]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [19]:
hotel_df.count()

City                   194
Country                194
Max Temp               194
Current Description    194
Lat                    194
Lng                    194
Hotel Name             194
dtype: int64

In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"


# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
7,12.0564,-61.7485
16,22.2855,114.1577
20,0.6000,-80.0333
25,5.5577,95.3222
26,-20.4081,57.7000
...,...,...
671,6.2106,124.1894
672,-10.1667,123.5833
673,8.1333,77.3000
677,-9.6658,-35.7353


In [25]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
              max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))